In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor

# === 1. Cargar en chunks y filtrar hasta dic 2019 ===
usecols = ["product_id", "periodo", "tn"]
chunks = pd.read_csv("top_30_corr.csv",
                     usecols=usecols,
                     parse_dates=["periodo"],
                     chunksize=500_000)

filtrados = []
total_rows = 0
chunk_count = 0

print("🟡 Procesando chunks...")
for chunk in chunks:
    chunk_count += 1
    original_rows = len(chunk)
    chunk = chunk[chunk["periodo"] <= "2019-12-01"]
    filtrados.append(chunk)
    total_rows += len(chunk)
    print(f"✅ Chunk {chunk_count}: {original_rows} filas → {len(chunk)} útiles (acumuladas: {total_rows})")

print("✅ Todos los chunks procesados. Total acumulado:", total_rows)

# === 2. Renombrar y preparar ===
df_train = pd.concat(filtrados, ignore_index=True)
df_train = df_train.rename(columns={"product_id": "item_id", "periodo": "timestamp", "tn": "target"})
df_train["item_id"] = df_train["item_id"].astype(str)

# Forzar timestamps al inicio de mes (por si hay algún desvío)
df_train["timestamp"] = df_train["timestamp"].dt.to_period("M").dt.to_timestamp()

# === 3. Crear enero 2020 sintético duplicando diciembre ===
print("📦 Generando enero sintético...")
enero_fake = df_train[df_train["timestamp"] == "2019-12-01"].copy()
enero_fake["timestamp"] = pd.to_datetime("2020-01-01")
enero_fake["target"] = pd.NA

# 🔁 Guardar enero_fake para inspección
enero_fake.to_csv("enero_fake_top30.csv", index=False)
print("📝 Archivo 'enero_fake_top30.csv' exportado para inspección.")

# === 4. Concatenar enero al dataset ===
df_train_con_enero = pd.concat([df_train, enero_fake], ignore_index=True)
df_train_con_enero = df_train_con_enero.sort_values(["item_id", "timestamp"])

# === 5. Entrenar modelo ===
print("⚙️ Entrenando modelo con AutoGluon...")
predictor = TimeSeriesPredictor(
    target="target",
    prediction_length=1,
    eval_metric="RMSE",
    freq="MS"  # ← CORRECCIÓN CLAVE
)
predictor.fit(train_data=df_train_con_enero, time_limit=600)
print("✅ Modelo entrenado.")

# === 6. Predecir febrero 2020 ===
print("🔮 Generando predicciones para febrero 2020...")
predicciones = predictor.predict(df_train_con_enero)

# === 7. Filtrar febrero 2020 ===
resultado = predicciones.reset_index()
print("🧪 Columnas disponibles en predicciones:", resultado.columns.tolist())

# Usamos la columna 'mean' como valor predicho
resultado = resultado[resultado["timestamp"] == pd.to_datetime("2020-02-01")]
resultado = resultado[["item_id", "mean"]].rename(columns={"item_id": "product_id", "mean": "tn_predicho"})

# === 8. Exportar resultado ===
resultado.to_csv("prediccion_feb2020_autogluon_top30.csv", index=False)
print("✅ Archivo exportado como 'prediccion_feb2020_autogluon_top30.csv'")



c:\Users\waldo\miniconda3\envs\autogluon-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🟡 Procesando chunks...
✅ Chunk 1: 500000 filas → 500000 útiles (acumuladas: 500000)
✅ Chunk 2: 405832 filas → 405832 útiles (acumuladas: 905832)
✅ Todos los chunks procesados. Total acumulado: 905832
📦 Generando enero sintético...
📝 Archivo 'enero_fake_top30.csv' exportado para inspección.


C:\Users\waldo\AppData\Local\Temp\ipykernel_15752\410407547.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_train_con_enero = pd.concat([df_train, enero_fake], ignore_index=True)
Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'c:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\AutogluonModels\ag-20250628_030622'


⚙️ Entrenando modelo con AutoGluon...


=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       39.11 GB / 63.68 GB (61.4%)
Disk Space Avail:   1350.52 GB / 1862.26 GB (72.5%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_data has 2220 rows (NaN fraction=2.7%), 60 time series. Median time series length is 37 (min=37, max=37). 

Provided data contains following columns:


✅ Modelo entrenado.
🔮 Generando predicciones para febrero 2020...


data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: SeasonalNaive


🧪 Columnas disponibles en predicciones: ['item_id', 'timestamp', 'mean', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
✅ Archivo exportado como 'prediccion_feb2020_autogluon_top30.csv'


In [4]:
import pandas as pd

# 1. Cargar predicciones de Autogluon (febrero 2020)
df_pred = pd.read_csv("prediccion_feb2020_autogluon_top30.csv")  # asumimos que tiene 'product_id' y 'tn_predicha'
total_predicho = df_pred["tn_predicho"].sum()

# 2. Cargar dataset base
df_real = pd.read_csv("top_30_corr.csv")
df_real["periodo"] = pd.to_datetime(df_real["periodo"])

# 3. Agrupar toneladas reales por período
tn_por_periodo = df_real.groupby("periodo")["tn"].sum().reset_index()

# 4. Agregar predicción de febrero 2020 como nueva fila
nueva_fila = pd.DataFrame({"periodo": [pd.Timestamp("2020-02-01")], "tn": [total_predicho]})
comparacion = pd.concat([tn_por_periodo, nueva_fila], ignore_index=True)

# 5. Ordenar por período
comparacion = comparacion.sort_values("periodo")

# 6. Mostrar
print(comparacion.tail(6))  # últimas filas para ver cómo se compara febrero 2020


      periodo           tn
31 2019-08-01  6763.956230
32 2019-09-01  9211.424580
33 2019-10-01  9392.195920
34 2019-11-01  9015.628160
35 2019-12-01  6586.565350
36 2020-02-01    19.698087
